<a href="https://www.kaggle.com/code/levii2375/us-airline-sentiment-analysis-project?scriptVersionId=164080676" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import numpy as np
import re
# from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/us-airline-sentiments/training_twitter_x_y_train.csv") #10980 rows 12 cols
test = pd.read_csv("/kaggle/input/us-airline-sentiments/test_twitter_x_test.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print(train['negativereason_gold'].nunique())
print(train['negativereason_gold'].value_counts(),"\n")

print(test.negativereason_gold.nunique())
print(test.negativereason_gold.value_counts())

In [ ]:
unique_value_counts = train['airline_sentiment'].value_counts()
print(unique_value_counts)

In [ ]:
import imblearn

In [ ]:
# from imblearn.over_sampling import RandomOverSampler

# # Assuming 'df' is your DataFrame and 'airline_sentiment' is the target column
# # Replace 'positive' and 'neutral' with your actual class names

# # Separate the dataset into features and target
# X = train.drop('airline_sentiment', axis=1)  # Features
# y = train['airline_sentiment']  # Target

# # Initialize RandomOverSampler
# oversampler = RandomOverSampler(sampling_strategy={'positive': 5000, 'neutral': 5000}, random_state=42)

# # Resample the dataset
# X_resampled, y_resampled = oversampler.fit_resample(X, y)

# # Combine the resampled data into a new DataFrame
# resampled_df = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='airline_sentiment')], axis=1)


In [ ]:
# unique_value_counts = y_resampled.value_counts()
# print(unique_value_counts)

## Cleaning

In [ ]:
drop_cols = ['airline_sentiment_gold','name','tweet_id', 'retweet_count','tweet_created','user_timezone','tweet_coord','tweet_location']
train.drop(drop_cols, axis = 1, inplace=True)
test.drop(drop_cols, axis = 1, inplace=True)

In [ ]:
# stops = stopwords.words('english')
stops = [
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", 
    "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 
    'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 
    'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 
    'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 
    'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 
    'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 
    'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 
    'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 
    'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 
    't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 
    've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', 
    "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', 
    "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', 
    "weren't", 'won', "won't", 'wouldn', "wouldn't"
]

print(stops)

In [ ]:
# stops = stopwords.words('english')
stops += list(punctuation)
stops += ['flight','airline','flights','AA']

In [ ]:
abbreviations = {'ppl': 'people','cust':'customer','serv':'service','mins':'minutes','hrs':'hours','svc': 'service',
           'u':'you','pls':'please'}

train_index = train[~train.negativereason_gold.isna()].index
test_index = test[~test.negativereason_gold.isna()].index

for index, row in train.iterrows():
    tweet = row.text
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet) #remove links
    tweet = re.sub('@[^\s]+','',tweet) #remove usernames
    tweet = re.sub('[\s]+', ' ', tweet) #remove additional whitespaces
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) #replace #word with word
    tweet = tweet.strip('\'"') #trim tweet
    words = []
    for word in tweet.split():    
#         if not hasNumbers(word):
        if word.lower() not in stops:
            if word in list(abbreviations.keys()):
                words.append(abbreviations[word])
            else:
                words.append(word.lower())   
    tweet = " ".join(words)
    tweet = " %s %s" % (tweet, row.airline)
    row.text = tweet
    if index in train_index:
        row.text = " %s %s" % (row.text, row.negativereason_gold)

for index, row in test.iterrows():
    tweet = row.text
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet) #remove links
    tweet = re.sub('@[^\s]+','',tweet) #remove usernames
    tweet = re.sub('[\s]+', ' ', tweet) #remove additional whitespaces
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) #replace #word with word
    tweet = tweet.strip('\'"') #trim tweet
    words = []
    for word in tweet.split(): 
#         if not hasNumbers(word):
        if word.lower() not in stops:
            if word in list(abbreviations.keys()):
                words.append(abbreviations[word])
            else:
                words.append(word.lower())
    tweet = " ".join(words)
    tweet = " %s %s" % (tweet, row.airline)
    row.text = tweet
    if index in test_index:
        row.text = " %s %s" % (row.text, row.negativereason_gold)

del train['negativereason_gold']
del test['negativereason_gold']

In [ ]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

for index, row in train.iterrows():
    row.text = deEmojify(row.text)

for index, row in test.iterrows():
    row.text = deEmojify(row.text)

In [ ]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

for index, row in train.iterrows():
    words = row.text.split()
    new_words = []
    for word in words:
        if not hasNumbers(word):
            new_words.append(word)
    row.text = " ".join(new_words)
    
for index, row in test.iterrows():
    words = row.text.split()
    new_words = []
    for word in words:
        if not hasNumbers(word):
            new_words.append(word)
    row.text = " ".join(new_words)

**Oversampling positive and neutral tweets**

In [ ]:
from imblearn.over_sampling import RandomOverSampler

# Assuming 'df' is your DataFrame and 'airline_sentiment' is the target column
# Replace 'positive' and 'neutral' with your actual class names

# Separate the dataset into features and target
X = train.drop('airline_sentiment', axis=1)  # Features
y = train['airline_sentiment']  # Target

# Initialize RandomOverSampler
oversampler = RandomOverSampler(sampling_strategy={'positive': 5000, 'neutral': 5000}, random_state=42)

# Resample the dataset
X_resampled, y_resampled = oversampler.fit_resample(X, y)

# Combine the resampled data into a new DataFrame
train = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='airline_sentiment')], axis=1)


In [ ]:
train.head()

In [ ]:

unique_value_counts = train['airline_sentiment'].value_counts()
print(unique_value_counts)


## Creating vocab and data formatting

In [ ]:
test.head()

In [ ]:
test['text'][3]

**model training using logistive regression**

***TFID vectorization to convert text to numbers***

In [ ]:
v = TfidfVectorizer(analyzer='word', max_features=3150, max_df = 0.8, ngram_range=(1,1))
train_features= v.fit_transform(train.text)
test_features=v.transform(test.text)

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming 'df' is your DataFrame and 'processed_text' is the column with processed text
X =  train_features
y = train['airline_sentiment']   # Replace 'target_column' with the actual target column name

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# Instantiate Logistic Regression model
clf = LogisticRegression(C=2.1, solver='liblinear', multi_class='auto')

# Fit the model on the training data
clf.fit(X_train,y_train)

# Make predictions on the test data
pred = clf.predict(X_test)
pred1 = clf.predict(X_train)

# Save predictions to a CSV file
predictions_df = pd.DataFrame({'Prediction': pred})
predictions_df.to_csv('predictions_twitter.csv', index=False)

# Evaluate the model
accuracy = accuracy_score(y_test, pred)
classification_rep = classification_report(y_test, pred)
confusion_mat = confusion_matrix(y_test, pred)

# Print evaluation metrics
print(f'Accuracy: {accuracy:.4f}')
print('\nClassification Report:\n', classification_rep)
print('\nConfusion Matrix:\n', confusion_mat)


In [ ]:
accuracy = accuracy_score(y_train, pred1)
classification_rep = classification_report(y_train, pred1)
confusion_mat = confusion_matrix(y_train, pred1)

# Print evaluation metrics
print(f'Accuracy: {accuracy:.4f}')
print('\nClassification Report:\n', classification_rep)
print('\nConfusion Matrix:\n', confusion_mat)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Predictions on the test set

# Compute the confusion matrix
cm_lr = confusion_matrix(y_test, pred)

# Plot the confusion matrix as a heatmap
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Neutral', 'Positive'], yticklabels=['Negative', 'Neutral', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix(logistic regression)')
plt.show()


***model training using SVM, using rbf kernel***

In [ ]:
clf = SVC(kernel="rbf", C= 0.9 , gamma = 'scale')
# clf = SVC(C = 1000, gamma = 0.001)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
pred1 = clf.predict(X_train)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, pred)
classification_rep = classification_report(y_test, pred)
confusion_mat = confusion_matrix(y_test, pred)

# Print evaluation metrics
print(f'Accuracy: {accuracy:.4f}')
print('\nClassification Report:\n', classification_rep)
print('\nConfusion Matrix:\n', confusion_mat)


In [ ]:
accuracy = accuracy_score(y_train, pred1)
classification_rep = classification_report(y_train, pred1)
confusion_mat = confusion_matrix(y_train, pred1)
# Print evaluation metrics
print(f'Accuracy: {accuracy:.4f}')
print('\nClassification Report:\n', classification_rep)
print('\nConfusion Matrix:\n', confusion_mat)

In [ ]:
with open('predictions_twitter2.csv', 'w') as f: #less accurate
    for item in pred:
        f.write("%s\n" % item)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Predictions on the test set

# Compute the confusion matrix
cm_svm = confusion_matrix(y_test, pred)
# Plot the confusion matrix as a heatmap
sns.heatmap(cm_svm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Neutral', 'Positive'], yticklabels=['Negative', 'Neutral', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix(SVM)')
plt.show()


**Hypertuning SVM model**

In [ ]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.svm import SVC

# # Split your data into training and testing sets
# # Define the SVM model

# svm_model = SVC()

# # Define the hyperparameters and their possible values for the grid search
# param_grid = {
#     'C': [0.1, 1, 10, 100],         # Regularization parameter
#     'kernel': ['linear', 'rbf', 'poly'],  # Kernel type
#     'gamma': [0.1, 1, 10],            # Kernel coefficient for 'rbf' and 'poly' kernels
#     'degree': [2, 3, 4],              # Degree of the polynomial kernel ('poly' only)
# }

# # Perform grid search with cross-validation
# grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5, scoring='accuracy')
# grid_search.fit(X_train, y_train)

# # Display the best hyperparameters
# best_params = grid_search.best_params_
# print("Best Hyperparameters:", best_params)

# # Get the best SVM model
# best_svm_model = grid_search.best_estimator_

# # Evaluate the model on the test set
# y_pred = best_svm_model.predict(X_test)

# # Display classification report or other evaluation metrics
# print(classification_report(y_test, y_pred))


In [ ]:
# v.get_feature_names()

In [ ]:
train

**simple neural network model**

In [ ]:
import tensorflow

In [ ]:
y = train['airline_sentiment']  # Map labels to numerical values

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


X =  train['text']
y = train['airline_sentiment'].map({'negative': 0, 'neutral': 1, 'positive': 2})   


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text
max_words = 10000  
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)


X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad sequences to have the same length
max_sequence_length = 100  # Adjust the maximum sequence length based on your data
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

# Print the processed data
print("X_train_padded shape:", X_train_padded.shape)
print("X_test_padded shape:", X_test_padded.shape)
print("X_train_padded example:", X_train_padded[0])

# Now, X_train_padded and X_test_padded are ready to be used for neural network training


In [ ]:
X_test_padded

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout

# Create a Sequential model
model = Sequential()

# Add an Embedding layer
embedding_dim = 16
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))

# Flatten the output of the Embedding layer
model.add(Flatten())
model.add(Dropout(0.2))

# Add a Dense hidden layer with ReLU activation
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.2))

# Add another Dense hidden layer with ReLU activation
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.2))

# Add a Dense output layer with softmax activation for multi-class classification
num_classes = 3
model.add(Dense(num_classes, activation='softmax'))

# Compile the model using the Adam optimizer and sparse categorical crossentropy loss
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

# Train the model on the training data
model.fit(X_train_padded, y_train, epochs=5, batch_size=32, validation_split=0.2)


In [ ]:
loss, accuracy = model.evaluate(X_train_padded, y_train)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy * 100:.2f}%')

In [ ]:
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy * 100:.2f}%')

**using LSTM layers in place of reLu activation layer**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


model2 = Sequential()

# Add an Embedding layer
embedding_dim = 16
model2.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))


model2.add(LSTM(units=64, return_sequences=True))
model2.add(Dropout(0.2))


model2.add(LSTM(units=32))
model2.add(Dropout(0.2))


num_classes = 3 
model2.add(Dense(num_classes, activation='softmax'))


model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


model2.summary()


model2.fit(X_train_padded, y_train, epochs=5, batch_size=32, validation_split=0.2)


In [ ]:
model2.summary()

In [ ]:
yprid = model2.predict(X_test_padded)

In [ ]:
loss, accuracy = model2.evaluate(X_train_padded, y_train)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy * 100:.2f}%')

In [ ]:
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy * 100:.2f}%')

In [ ]:
#now let's try it now on a completely new data

In [ ]:
Xtest =  test['text']
#ytest = test['airline'].map({'negative': 0, 'neutral': 1, 'positive': 2})   # Replace 'target_column' with the actual target column name

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
max_words = 10000  # Adjust the maximum number of words to consider based on your data
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(Xtest)

# Convert text to sequences of integers
X_test_sequences = tokenizer.texts_to_sequences(Xtest)

# Pad sequences to have the same length
max_sequence_length = 100  # Adjust the maximum sequence length based on your data
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
#X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

In [ ]:
yprid_test = model.predict(X_test_padded)

In [ ]:
yprid_test